In [1]:
import json
import pandas as pd

In [2]:
df = pd.read_csv('operators_data.csv')

In [3]:
df.head()

,operator_address,operator_id,percentage,quorum_id,total_batches,total_unsigned_batches,timestamp
0,0x00107cfdeaddc0a3160ed2f6fedd627f313e7b1a,0x57309111be2293fbcfd77e8479a4f2570f8956aae8cd...,0.0000,0,144,0,1717934400
1,0x006b988f89579e5842bcd029955dfbfc334b6826,0x852f5c1ecf96472bb48fc8e9373b40c74ac3e2cdf32a...,0.0000,1,144,0,1717934400
2,0x01a7c2568693d65a367fde016b48c63f6673d4dc,0x7e06688af02ac562ee27c843b7e688eb1c67b469b3e6...,0.0000,1,144,0,1717934400
3,0x033bfb405e809a303df875a6e018f1a64e5dbae9,0x2e2ee953aa3c2a499640270eaaf2ba1973265f7e0d0b...,0.0625,0,144,9,1717934400
4,0x033bfb405e809a303df875a6e018f1a64e5dbae9,0x2e2ee953aa3c2a499640270eaaf2ba1973265f7e0d0b...,0.0625,1,144,9,1717934400


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13181 entries, 0 to 13180
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   operator_address        13181 non-null  object 
 1   operator_id             13181 non-null  object 
 2   percentage              13181 non-null  float64
 3   quorum_id               13181 non-null  int64  
 4   total_batches           13181 non-null  int64  
 5   total_unsigned_batches  13181 non-null  int64  
 6   timestamp               13181 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 721.0+ KB


In [5]:
# change operator address to string
df['operator_address'] = df['operator_address'].astype(str)
df['operator_id'] = df['operator_id'].astype(str)

In [6]:
len(df), len(df['operator_address'].unique())

(13181, 263)

In [7]:
# drop rows in which operator_address and timestamp are same and quorum_id is 1. keep the one in which quoram_id is 0
df = df.drop_duplicates(subset=['operator_address', 'timestamp'], keep='first')

In [8]:
len(df), len(df['operator_address'].unique())

(9732, 263)

In [9]:
# sum of all percentage of a certain operator
df['total_percentage'] = df.groupby('operator_address')['percentage'].transform('sum')

# frequency of each operator
df['frequency'] = df.groupby('operator_address')['operator_address'].transform('count')

In [10]:
df.head()

,operator_address,operator_id,percentage,quorum_id,total_batches,total_unsigned_batches,timestamp,total_percentage,frequency
0,0x00107cfdeaddc0a3160ed2f6fedd627f313e7b1a,0x57309111be2293fbcfd77e8479a4f2570f8956aae8cd...,0.0000,0,144,0,1717934400,0.019536,52
1,0x006b988f89579e5842bcd029955dfbfc334b6826,0x852f5c1ecf96472bb48fc8e9373b40c74ac3e2cdf32a...,0.0000,1,144,0,1717934400,0.010856,17
2,0x01a7c2568693d65a367fde016b48c63f6673d4dc,0x7e06688af02ac562ee27c843b7e688eb1c67b469b3e6...,0.0000,1,144,0,1717934400,0.007636,29
3,0x033bfb405e809a303df875a6e018f1a64e5dbae9,0x2e2ee953aa3c2a499640270eaaf2ba1973265f7e0d0b...,0.0625,0,144,9,1717934400,0.267125,20
5,0x047438c5ceaa6d47e8691b51f5b34c6d41a00e3d,0xdbf03be4c90917ec42220446cee2b1226ac96fd074c5...,0.0000,1,144,0,1717934400,0.000000,29


In [11]:
# order by timestamp
df = df.sort_values(by='timestamp')

In [12]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the CSV file
# df = pd.read_csv('operator_data.csv')

# Function to create lag features
def create_lag_features(df, lag=1):
    for i in range(1, lag + 1):
        df[f'lag_{i}'] = df['percentage'].shift(i)
    return df

# Create lag features for the past 5 days
df = df.groupby('operator_address').apply(create_lag_features, lag=7)
df = df.dropna().reset_index(drop=True)
# print(df.head())
 
# Define features and target
# features = [col for col in df.columns if 'lag_' in col]
features = ['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'quorum_id', 'total_batches', 'total_unsigned_batches', 'frequency', 'total_percentage', 'timestamp']
target = 'percentage'

# Split the data into training and testing sets
train_df = df[df['timestamp'] < 1717588800]
test_df = df[df['timestamp'] >= 1717588800]

X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]

# Increase the number of trees and maximum depth
n_estimators = 2  # Increase the number of trees
max_depth = 6  # Increase the maximum depth of each tree

# Initialize and train the XGBoost model
model = xgb.XGBRegressor(n_estimators=n_estimators, max_depth=max_depth)
model.fit(X_train, y_train)

/var/folders/pj/vyqltmgj79xghgpb38_96m040000gn/T/ipykernel_64104/137865981.py:17: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('operator_address').apply(create_lag_features, lag=7)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=2, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [13]:
len(X_test), len(X_train), len(df)

(1258, 6654, 7912)

In [14]:
from sklearn.metrics import mean_squared_error as MSE
# Predict the model 
pred = model.predict(X_test)
# RMSE Computation 
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))

RMSE :  0.120531


In [15]:
pred

array([0.02207324, 0.02207324, 0.02207324, ..., 0.02207324, 0.02207324,
       0.02207324], dtype=float32)

In [16]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Define the initial types for the ONNX model
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]

# Convert the scikit-learn model to ONNX
onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save the ONNX model to a file
with open("xgb_operator.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

MissingShapeCalculator: Unable to find a shape calculator for type '<class 'xgboost.sklearn.XGBRegressor'>'.
It usually means the pipeline being converted contains a
transformer or a predictor with no corresponding converter
implemented in sklearn-onnx. If the converted is implemented
in another library, you need to register
the converted so that it can be used by sklearn-onnx (function
update_registered_converter). If the model is not yet covered
by sklearn-onnx, you may raise an issue to
https://github.com/onnx/sklearn-onnx/issues
to get the converter implemented or even contribute to the
project. If the model is a custom model, a new converter must
be implemented. Examples can be found in the gallery.
